In [1]:
!pip install prefect==2.12.1 transformers pyspark kagglehub openpyxl griffe==0.39.1 email-validator -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB

In [2]:
import os
import kagglehub
import pandas as pd
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import concat_ws, lower, regexp_replace
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from transformers import pipeline
from prefect import flow, task, get_run_logger

# -------------------------
# Constants / Paths
# -------------------------
DATA_LAKE_TRAIN = "/content/data_lake/train"
DATA_LAKE_TEST = "/content/data_lake/test"
DATA_LAKE_TRAIN_SENTIMENT = "/content/data_lake/train_with_sentiment_sample"
SENTIMENT_EXCEL = "/content/data_lake/train_with_sentiment_sample.xlsx"


In [3]:
from google.colab import files
files.upload()  # upload your kaggle.json

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


Saving kaggle.json to kaggle.json


In [4]:
# Spark session
def get_spark_session():
    return SparkSession.builder \
        .appName("AmazonReviewsPipeline") \
        .config("spark.sql.shuffle.partitions", "4") \
        .getOrCreate()

spark = get_spark_session()


In [5]:
@task(retries=3, retry_delay_seconds=30)
def download_dataset():
    logger = get_run_logger()
    logger.info("Downloading dataset from Kaggle...")
    path = os.path.expanduser("~/amazon_reviews_dataset")
    os.makedirs(path, exist_ok=True)

    dataset_path = kagglehub.dataset_download("kritanjalijain/amazon-reviews")
    logger.info(f"Dataset downloaded at {dataset_path}")
    return dataset_path


In [6]:
@task
def load_and_clean_data(dataset_path):
    logger = get_run_logger()
    logger.info("Loading and cleaning data...")

    train_file = f"{dataset_path}/train.csv"
    test_file = f"{dataset_path}/test.csv"

    columns = ['label', 'title', 'review']

    # Load train and test
    train_sdf = spark.read.csv(train_file, header=False, sep=",", quote='"', escape='\\',
                               multiLine=True, inferSchema=True).toDF(*columns)
    test_sdf = spark.read.csv(test_file, header=False, sep=",", quote='"', escape='\\',
                              multiLine=True, inferSchema=True).toDF(*columns)

    # Quick checks
    train_sdf.show(5, truncate=100)
    train_sdf.printSchema()
    train_sdf.groupBy('label').count().show()


    # Combine title + review and clean
    train_sdf = train_sdf.withColumn("full_review", concat_ws(" ", train_sdf.title, train_sdf.review))
    train_sdf = train_sdf.withColumn("full_review", lower(regexp_replace("full_review", "[^a-zA-Z ]", " ")))

    test_sdf = test_sdf.withColumn("full_review", concat_ws(" ", test_sdf.title, test_sdf.review))
    test_sdf = test_sdf.withColumn("full_review", lower(regexp_replace("full_review", "[^a-zA-Z ]", " ")))


    print("Train rows:", train_sdf.count())
    train_sdf.printSchema()

    # Clear Spark cache
    spark.catalog.clearCache()


    # Save cleaned data
    os.makedirs(DATA_LAKE_TRAIN, exist_ok=True)
    os.makedirs(DATA_LAKE_TEST, exist_ok=True)


    train_sdf.write.mode('overwrite').parquet(DATA_LAKE_TRAIN)
    test_sdf.write.mode('overwrite').parquet(DATA_LAKE_TEST)

    # Explicitly read back the data to ensure it's written and visible
    spark.read.parquet(DATA_LAKE_TRAIN).count()
    spark.read.parquet(DATA_LAKE_TEST).count()

    import time
    time.sleep(3)  # give Spark time to flush writes

    print("Cleaned Parquet files saved.")

    logger.info("Data cleaned and saved to parquet.")

    tokenizer = Tokenizer(inputCol="full_review", outputCol="words")
    remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")

    train_sdf = tokenizer.transform(train_sdf)
    train_sdf = remover.transform(train_sdf)

    test_sdf = tokenizer.transform(test_sdf)
    test_sdf = remover.transform(test_sdf)

    train_sdf.write.mode('overwrite').parquet(DATA_LAKE_TRAIN)
    test_sdf.write.mode('overwrite').parquet(DATA_LAKE_TEST)

    logger.info("Feature engineering completed.")


    return DATA_LAKE_TRAIN, DATA_LAKE_TEST

In [8]:
@task
def run_sentiment_analysis(train_path, sample_size=3000):
    logger = get_run_logger()
    logger.info("Running sentiment analysis...")

    train_sdf = spark.read.parquet(train_path)

    # Hugging Face pipeline
    sentiment_model = pipeline(
        "sentiment-analysis",
        model="distilbert-base-uncased-finetuned-sst-2-english",
        device=0  # GPU
    )

    sample_df = train_sdf.limit(sample_size)
    reviews = [row.full_review for row in sample_df.collect()]

    batch_size = 32
    results = []
    for i in range(0, len(reviews), batch_size):
        batch = reviews[i:i+batch_size]
        results.extend(sentiment_model(batch))

    rows = [Row(full_review=r, hf_sentiment=res['label']) for r, res in zip(reviews, results)]
    train_sdf_sentiment = spark.createDataFrame(rows)

    # Save parquet
    train_sdf_sentiment.write.mode('overwrite').parquet(DATA_LAKE_TRAIN_SENTIMENT)

    # Save Excel
    pd_df = train_sdf_sentiment.toPandas()
    pd_df.to_excel(SENTIMENT_EXCEL, index=False)
    logger.info(f"✅ Sentiment Excel saved at {SENTIMENT_EXCEL}")

    return DATA_LAKE_TRAIN_SENTIMENT, SENTIMENT_EXCEL


In [9]:
@flow(name="Amazon Reviews Pipeline")
def amazon_reviews_pipeline():
    dataset_path = download_dataset()
    train_path, test_path = load_and_clean_data(dataset_path)
    sentiment_parquet, sentiment_excel = run_sentiment_analysis(train_path)
    return sentiment_parquet, sentiment_excel


In [10]:
sentiment_parquet, sentiment_excel = amazon_reviews_pipeline()
print(f"Pipeline completed! Parquet: {sentiment_parquet}, Excel: {sentiment_excel}")


/usr/lib/python3.12/contextlib.py:144: SAWarning: Skipped unsupported reflection of expression-based index ix_flow_run__coalesce_start_time_expected_start_time_desc
  next(self.gen)
/usr/lib/python3.12/contextlib.py:144: SAWarning: Skipped unsupported reflection of expression-based index ix_flow_run__coalesce_start_time_expected_start_time_asc
  next(self.gen)


09:08:14.113 | INFO    | prefect.engine - Created flow run 'tireless-dove' for flow 'Amazon Reviews Pipeline'

09:08:14.366 | INFO    | Flow run 'tireless-dove' - Created task run 'download_dataset-0' for task 'download_dataset'

09:08:14.369 | INFO    | Flow run 'tireless-dove' - Executing 'download_dataset-0' immediately...

09:08:14.492 | INFO    | Task run 'download_dataset-0' - Downloading dataset from Kaggle...

...

Device set to use cpu


09:23:22.638 | INFO    | Task run 'run_sentiment_analysis-0' - ✅ Sentiment Excel saved at /content/data_lake/train_with_sentiment_sample.xlsx

09:23:22.695 | INFO    | Task run 'run_sentiment_analysis-0' - Finished in state Completed()

09:23:22.728 | INFO    | Flow run 'tireless-dove' - Finished in state Completed()

Pipeline completed! Parquet: /content/data_lake/train_with_sentiment_sample, Excel: /content/data_lake/train_with_sentiment_sample.xlsx
